In [11]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('shared_articles.csv')
#Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?
df1 = df1[df1['eventType'] =='CONTENT SHARED']
df1.shape[0]
df2 = pd.read_csv('users_interactions.csv')
#Давайте предварительно преобразуем столбцы personId, contentId в таблицах к строкам. Это преобразование пригодится нам в дальнейшем
df2.personId = df2.personId.astype(str)
df2.contentId = df2.contentId.astype(str)
df2.contentId = df2.contentId.astype(str)
#В колонке eventType описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:
#VIEW — просмотр,
#LIKE — лайк,
#COMMENT CREATED — комментарий,
#FOLLOW — подписка,
#BOOKMARK — добавление в закладки.
#Однако у нас есть информация о различных действиях пользователя,
# и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
#Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами).
# Вычислите среднее значение для полученного признака. 
df2['eventStrength'] = df2.eventType.apply(lambda x: event_type[x])
df2['eventStrength'].mean()

#оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями
users_interactions_count_df = (
    df2
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

#print(len(users_with_enough_interactions_df))

#Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей
# (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?
interactions_from_selected_users_df = df2.loc[np.in1d(df2.personId,
            users_with_enough_interactions_df)]
print(interactions_from_selected_users_df.shape)

(69868, 9)


Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

def smooth_user_preference(x):
    return math.log(1+x, 2)

Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

In [12]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df['last_timestamp'].mean()

1470605340.0403006

Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. Сколько объектов попало в обучающую выборку?

In [13]:
split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(len(interactions_train_df))

29325


Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.

In [14]:
final_df = (
    interactions_train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    interactions_test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df.head()

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [15]:
popular = (
    interactions_train_df
    .groupby('contentId')
    .eventStrength.sum().reset_index()
    .sort_values('eventStrength', ascending=False)
    .contentId.values
)
popular[0]

'-6783772548752091658'

Теперь необходимо сформировать рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.

Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.

Для вычисления precision@10 воспользуйтесь следующей функцией:

In [16]:
top_k = 10
 
final_df['popular'] = (
    final_df.true_train
    .apply(
        lambda x:
        popular[~np.in1d(popular, x)][:top_k]
    )
)
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()
calc_precision('popular')

0.006454207722621089

Примечание. Стоит отметить, что качество РС оценивается не так, как в задачах классификации: показателей выше 0.5 добиться практически невозможно, и даже результат 0.1–0.2 — индикатор высокого качества.